In [ ]:
# instantiate a model from src

from hydra_zen import instantiate

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils import parameters_to_vector


# 모델 정의
class SimpleModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Sequential(nn.Linear(10, 3), nn.ReLU(), nn.Linear(3, 1))

    def forward(self, x):
        return self.linear(x)


# 코사인 유사도 계산 함수
def cosine_similarity(grads1, grads2):
    vec1 = parameters_to_vector(grads1)
    vec2 = parameters_to_vector(grads2)
    return torch.nn.functional.cosine_similarity(vec1.unsqueeze(0), vec2.unsqueeze(0), dim=1)


# 모델 인스턴스화
model1 = SimpleModel()
model2 = SimpleModel()

# 옵티마이저 정의 (모델1에 대해서만)
optimizer = optim.SGD(model1.parameters(), lr=0.01)
optimizer2 = optim.SGD(model2.parameters(), lr=0.01)
# 임의의 데이터 생성 (10차원 입력, 1차원 출력)
x = torch.randn(64, 10)
y = torch.randn(64, 1)

similarity_list = []
for idx, iteration in enumerate(range(2000)):
    optimizer.zero_grad()  # 그래디언트 초기화

    # 두 모델의 예측 및 손실 계산
    pred1 = model1(x)
    pred2 = model2(x)
    loss1 = (pred1 - y).pow(2).mean()
    loss2 = (pred2 - y).pow(2).mean()
    (
        print(f"Iteration {iteration}: Loss1 = {loss1.item()}, Loss2 = {loss2.item()}")
        if idx % 100 == 0
        else None
    )
    # 그래디언트 계산
    loss1.backward()
    loss2.backward()

    # 그래디언트 코사인 유사도 계산
    grads1 = [p.grad.detach().clone() for p in model1.parameters()]
    grads2 = [p.grad for p in model2.parameters()]
    similarity_list.append(cosine_similarity(grads1, grads2))
    # print(f"Iteration {iteration}: Cosine Similarity = {similarity.item()}")
    # 모델1의 파라미터 업데이트
    optimizer.step()
    for p, grad in zip(model2.parameters(), grads1):
        p.grad = grad
    optimizer2.step()

In [ ]:
# plot the cosine similarity
import matplotlib.pyplot as plt

plt.plot(similarity_list)
plt.xlabel("Iteration")
plt.ylabel("Cosine Similarity")
plt.title("Cosine Similarity between Model1 and Model2 Gradients")
plt.show()